In [17]:
import requests
from bs4 import BeautifulSoup 
import os
from dotenv import load_dotenv
import nltk
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob
import string
import re
import matplotlib.pyplot as plt

nltk.download('punkt_tab')
nltk.download("punkt")
nltk.download("stopwords")
nltk.data.path.append('C:\\Users\\Xavier Lee\\AppData\\Roaming\\nltk_data')


[nltk_data] Downloading package punkt_tab to C:\Users\Xavier
[nltk_data]     Lee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Xavier
[nltk_data]     Lee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Xavier
[nltk_data]     Lee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
load_dotenv()  # Load .env variables

API_KEY = os.getenv("NEWS_API_KEY")

if not API_KEY:
    raise ValueError("API key not found. Make sure you have a .env file with NEWS_API_KEY set.")


QUERY = "AI market trends"
URL = f"https://newsapi.org/v2/everything?q={QUERY}&apiKey={API_KEY}"


In [25]:
def get_news():
    response = requests.get(URL)
    data = response.json()
    
    if data.get("status") != "ok":
        print("Error fetching news:", data)
        return []
    
    return [{"title": article["title"], "url": article["url"], "source": article["source"]["name"]}
            for article in data.get("articles", [])]


In [27]:
# Example usage
news_articles = get_news()
for article in news_articles:
    print(article["title"], "-", article["url"])

We asked OpenAI’s o1 about the top AI trends in 2025 — here’s a look into our conversation - https://venturebeat.com/ai/we-asked-openais-o1-about-the-top-ai-trends-in-2025-heres-a-look-into-our-conversation/
Samsung Galaxy S25 Ultra vs. OnePlus 13: A battle of big, powerful Android phones - https://www.androidcentral.com/phones/samsung-galaxy-s25-ultra-vs-oneplus-13
What Trump’s Executive Orders Really Mean for the Climate - https://time.com/7208857/trump-oil-executive-orders-climate-impact/
The 'AI game continues,' IG CEO says - https://finance.yahoo.com/video/ai-game-continues-ig-ceo-164351619.html
Is BigCommerce Holdings Inc. (BIGC) the Best Tech Stock to Buy Right Now Under $10? - https://finance.yahoo.com/news/bigcommerce-holdings-inc-bigc-best-144716634.html
Coca-Cola Company (KO): Among the Best Widow and Orphan Stocks to Invest In - https://finance.yahoo.com/news/coca-cola-company-ko-among-153440511.html
Sam Altman says ChatGPT 4o is the 'best search product on the web' in a ch

In [28]:
def extract_keywords(news_articles):
    """ Extracts top 10 trending keywords from news headlines """
    text = " ".join(article["title"] for article in news_articles)
    words = word_tokenize(text.lower())

    # Remove stopwords and punctuation
    stop_words = set(stopwords.words("english") + list(string.punctuation))
    words = [word for word in words if word not in stop_words and word.isalpha()]

    # Count word frequency
    word_freq = Counter(words)
    return word_freq.most_common(10)

In [ ]:
def analyze_sentiment(news_articles):
    """ Performs sentiment analysis on news headlines """
    sentiment_scores = {"positive": 0, "neutral": 0, "negative": 0}

    for article in news_articles:
        analysis = TextBlob(article["title"])
        sentiment = analysis.sentiment.polarity

        if sentiment > 0:
            sentiment_scores["positive"] += 1
        elif sentiment < 0:
            sentiment_scores["negative"] += 1
        else:
            sentiment_scores["neutral"] += 1

    total = sum(sentiment_scores.values())
    sentiment_percentages = {k: round((v / total) * 100, 2) for k, v in sentiment_scores.items()} if total > 0 else sentiment_scores

    return sentiment_percentages

In [30]:
nltk.data.find('tokenizers/punkt')

FileSystemPathPointer('C:\\Users\\Xavier Lee\\AppData\\Roaming\\nltk_data\\tokenizers\\punkt')

In [31]:
if news_articles:
    print("\nLatest AI News:")
    for article in news_articles:
        print(f"{article['title']} - {article['url']}")

    # Extract trending keywords
    keywords = extract_keywords(news_articles)
    print("\nTrending Keywords:", keywords)

    # Perform sentiment analysis
    sentiment_results = analyze_sentiment(news_articles)
    print("\nSentiment Analysis:", sentiment_results)
else:
    print("No news articles found.")


Latest AI News:
We asked OpenAI’s o1 about the top AI trends in 2025 — here’s a look into our conversation - https://venturebeat.com/ai/we-asked-openais-o1-about-the-top-ai-trends-in-2025-heres-a-look-into-our-conversation/
Samsung Galaxy S25 Ultra vs. OnePlus 13: A battle of big, powerful Android phones - https://www.androidcentral.com/phones/samsung-galaxy-s25-ultra-vs-oneplus-13
What Trump’s Executive Orders Really Mean for the Climate - https://time.com/7208857/trump-oil-executive-orders-climate-impact/
The 'AI game continues,' IG CEO says - https://finance.yahoo.com/video/ai-game-continues-ig-ceo-164351619.html
Is BigCommerce Holdings Inc. (BIGC) the Best Tech Stock to Buy Right Now Under $10? - https://finance.yahoo.com/news/bigcommerce-holdings-inc-bigc-best-144716634.html
Coca-Cola Company (KO): Among the Best Widow and Orphan Stocks to Invest In - https://finance.yahoo.com/news/coca-cola-company-ko-among-153440511.html
Sam Altman says ChatGPT 4o is the 'best search product on

In [14]:
AI_COMPANIES = ["OpenAI", "Google", "Microsoft", "NVIDIA", "Meta", "Amazon", "Anthropic", "Tesla", "IBM", "Apple"]

def extract_companies(news_articles):
    """ Finds frequently mentioned AI companies in news headlines """
    company_mentions = {company: 0 for company in AI_COMPANIES}

    for article in news_articles:
        for company in AI_COMPANIES:
            if company.lower() in article["title"].lower():
                company_mentions[company] += 1

    return {k: v for k, v in sorted(company_mentions.items(), key=lambda item: item[1], reverse=True) if v > 0}


In [15]:
def calculate_ai_market_score(sentiment_results, keywords, company_trends):
    """ Calculates AI Market Score based on sentiment, keywords, and company mentions """
    positive_sentiment = sentiment_results.get("positive", 0)
    keyword_mentions = sum(dict(keywords).values())
    company_mentions = sum(company_trends.values())

    # Normalize values to fit the score system
    sentiment_weight = positive_sentiment * 0.5  # 50% weight
    keyword_weight = min(keyword_mentions, 100) * 0.3  # 30% weight, cap at 100
    company_weight = min(company_mentions, 50) * 0.2  # 20% weight, cap at 50

    ai_market_score = round(sentiment_weight + keyword_weight + company_weight, 2)
    return ai_market_score


In [16]:
# Example usage
news_articles = get_news()

if news_articles:
    print("\n🔹 Latest AI News:")
    for article in news_articles[:10]:  # Show first 10 results
        print(f"➡ {article['title']} - {article['url']}")

    # Extract trending keywords
    keywords = extract_keywords(news_articles)
    print("\n🔥 Trending AI Keywords:", keywords)

    # Perform sentiment analysis
    sentiment_results = analyze_sentiment(news_articles)
    print("\n📈 Sentiment Analysis:", sentiment_results)

    # Extract AI companies mentioned
    company_trends = extract_companies(news_articles)
    print("\n🏢 AI Companies Mentioned in News:", company_trends)

    # Calculate AI Market Score
    ai_market_score = calculate_ai_market_score(sentiment_results, keywords, company_trends)
    print("\n💹 AI Market Score:", ai_market_score)

    # Provide Investment Insight
    if ai_market_score > 80:
        print("🚀 AI Market is HOT! This could be a good time to invest.")
    elif 50 <= ai_market_score <= 79:
        print("⚖️ AI Market is stable. Hold investments or proceed cautiously.")
    else:
        print("❌ AI Market is cooling down. Consider reducing investments.")
else:
    print("No news articles found.")


🔹 Latest AI News:
➡ We asked OpenAI’s o1 about the top AI trends in 2025 — here’s a look into our conversation - https://venturebeat.com/ai/we-asked-openais-o1-about-the-top-ai-trends-in-2025-heres-a-look-into-our-conversation/
➡ Samsung Galaxy S25 Ultra vs. OnePlus 13: A battle of big, powerful Android phones - https://www.androidcentral.com/phones/samsung-galaxy-s25-ultra-vs-oneplus-13
➡ What Trump’s Executive Orders Really Mean for the Climate - https://time.com/7208857/trump-oil-executive-orders-climate-impact/
➡ The 'AI game continues,' IG CEO says - https://finance.yahoo.com/video/ai-game-continues-ig-ceo-164351619.html
➡ Is BigCommerce Holdings Inc. (BIGC) the Best Tech Stock to Buy Right Now Under $10? - https://finance.yahoo.com/news/bigcommerce-holdings-inc-bigc-best-144716634.html
➡ Coca-Cola Company (KO): Among the Best Widow and Orphan Stocks to Invest In - https://finance.yahoo.com/news/coca-cola-company-ko-among-153440511.html
➡ Sam Altman says ChatGPT 4o is the 'best s